In [24]:
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
import time

from tqdm import tqdm
from sklearn.linear_model import LogisticRegression


In [89]:
def trim_searchid(ids, booking_bools):
    '''
    removes searchid's where no hotel was booked from training data
    '''
    no_booking = []
    booking = False
    
    for i in tqdm(range(len(ids))):
        
        if booking_bools[i] == 1:
            booking = True
        # calculate relative price to average price
        if ids[i-1] != ids[i] or i == len(ids)-1: 
            if booking == False:
                no_booking.append(ids[i-1])
            booking = False

        
    
    return no_booking

In [95]:
# import datasets
test = pd.read_csv("../test.csv")
train = pd.read_csv("../train2.csv")

# hier verwijder ik deze column, kunnen ook proberen met nan op 0 zetten.
train = train.drop(columns=['srch_query_affinity_score'])
test = test.drop(columns=['srch_query_affinity_score'])


In [93]:
'''
werkt nog niet!
'''
ids = train['srch_id'].to_numpy()
booking_bools = train['booking_bool'].to_numpy()
ids_to_be_removed = trim_searchid(ids, booking_bools)

print(len(train.index))

for id in ids_to_be_removed:
    to_be_removed = train[training_subset['srch_id'] ==id].index
    train.drop(to_be_removed , inplace=True)
    
print(len(train.index))

100%|██████████| 4958347/4958347 [00:06<00:00, 781454.93it/s]
<ipython-input-93-a2d7330f1b47>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  to_be_removed = train[training_subset['srch_id'] ==index].index


4958347


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [ ]:
search_id = train['srch_id']
prop_id = train['prop_id']
click_bool = train['click_bool']
booking_bool = train['booking_bool']
position = train['position']
# gross_bookings_usd = train['gross_bookings_usd']

In [39]:
results = pd.DataFrame()
results['SearchId'] = search_id
results['PropertyId'] = prop_id
results['Booking_bool'] = booking_bool
results['Probability_of_booking'] = [0] * len(train)
results

,SearchId,PropertyId,Booking_bool,Probability_of_booking
0,1,893,0,0
1,1,10404,0,0
2,1,21315,0,0
3,1,27348,0,0
4,1,29604,0,0
...,...,...,...,...
4958342,332785,77700,0,0
4958343,332785,88083,0,0
4958344,332785,94508,0,0
4958345,332785,128360,1,0


In [40]:
training_subset = train[:50000]
testing_subset = train[50000:60000]

training_results_subset = results[:50000]
testing_results_subset = results[50000:60000]


In [41]:
training_subset.isnull().values.any()
check_for_nan = training_subset.isnull().sum()
print(check_for_nan)

Unnamed: 0                     0
Unnamed: 0.1                   0
Unnamed: 0.1.1                 0
Unnamed: 0.1.1.1               0
srch_id                        0
site_id                        0
visitor_location_country_id    0
visitor_hist_starrating        0
visitor_hist_adr_usd           0
prop_country_id                0
prop_id                        0
prop_starrating                0
prop_review_score              0
prop_brand_bool                0
prop_location_score1           0
prop_log_historical_price      0
price_usd                      0
promotion_flag                 0
srch_destination_id            0
srch_length_of_stay            0
srch_booking_window            0
srch_adults_count              0
srch_children_count            0
srch_room_count                0
srch_saturday_night_bool       0
orig_destination_distance      0
random_bool                    0
comp_rate                      0
comp_inv                       0
comp_percent                   0
month     

In [88]:
clf = LogisticRegression(max_iter=1000).fit(training_subset, np.ravel(training_results_subset['Booking_bool']))


ValueError: Found input variables with inconsistent numbers of samples: [33467, 50000]

In [49]:
predictions = clf.predict_proba(testing_subset)
testing_results_subset['Probability_of_booking'] = pred_1
testing_results_subset

<ipython-input-49-12d906feb615>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing_results_subset['Probability_of_booking'] = pred_1


,SearchId,PropertyId,Booking_bool,Probability_of_booking
50000,3314,78605,0,0.034486
50001,3314,82035,0,0.019272
50002,3314,92123,0,0.002899
50003,3314,105713,0,0.007656
50004,3314,112415,0,0.140877
...,...,...,...,...
59995,3997,67140,0,0.001672
59996,3997,67619,0,0.003699
59997,3997,72508,0,0.001447
59998,3997,77444,0,0.003595


In [48]:
pred_1 = []
for i in range(len(predictions)):
    pred_1.append(predictions[i][1])
    
testing_results_subset['Probability_of_booking'] = pred_1

<ipython-input-48-da499298fc8b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing_results_subset['Probability_of_booking'] = pred_1


In [65]:
final_results = testing_results_subset.sort_values(by = ['SearchId','Probability_of_booking'], ascending=[True,False])
ids = final_results['SearchId'].to_numpy()
print(len(set(ids)))
print(len(final_results[final_results['Booking_bool']==1]))
# print(final_results)


409
284


## Hele dateset:

In [99]:
training_subset = train[:4500000]
testing_subset = train[4500000:]

training_results_subset = results[:4500000]
testing_results_subset = results[4500000:]


clf = LogisticRegression(max_iter=1000).fit(training_subset, np.ravel(training_subset['booking_bool']))


predictions = clf.predict_proba(testing_subset)

pred_1 = []
for i in range(len(predictions)):
    pred_1.append(predictions[i][1])
    
testing_results_subset['Probability_of_booking'] = pred_1
testing_results_subset

<ipython-input-99-30fbc5852b38>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing_results_subset['Probability_of_booking'] = pred_1


,SearchId,PropertyId,Booking_bool,Probability_of_booking
4500000,301910,37278,0,0.006411
4500001,301910,38183,0,0.006378
4500002,301910,40937,0,0.006321
4500003,301910,50188,0,0.005800
4500004,301910,57905,0,0.005325
...,...,...,...,...
4958342,332785,77700,0,0.004801
4958343,332785,88083,0,0.004378
4958344,332785,94508,0,0.004135
4958345,332785,128360,1,0.003074
